In [ ]:
import boto3, json

model_ids = ["us.meta.llama3-2-90b-instruct-v1:0", 
            "us.meta.llama3-1-70b-instruct-v1:0", 
            "us.meta.llama3-2-3b-instruct-v1:0", 
            "meta.llama3-1-8b-instruct-v1:0"
]

def runtime_prompt_and_response(model_id, prompt):
    # Initialize the Bedrock client
    bedrock_client = boto3.client('bedrock-runtime')

    # Specify the model and input
    response = bedrock_client.invoke_model(
        modelId = model_id ,  # Replace with your model ID
        contentType="application/json",
        accept="application/json",
        body=json.dumps({
            "prompt": prompt,
            "temperature":1.0,

        })
    )

    # Parse the response
    result = json.load(response['body'])
    print(result['generation'])
    return result


In [ ]:
outputs = []

In [ ]:
scenario = """A 72-year-old female patient has post-operative, acute blood loss anemia following
 coronary artery bypass x3 for coronary artery disease (CAD) with refractory angina. 
 She is taking Ferrous Sulfate and skilled nursing will monitor CBC weekly and 
 perform wound care. She has hypertension (HTN) and congestive heart failure (CHF) 
 which are both stable. The focus of the home health admission is surgical aftercare."""

text= f'''
The following is a report of a home health patient: 
===
{scenario}
===
Provide the following: 
- Bulletpoint list of relevant conditions 
- Primary focus of care for home health 
- Adhering to ICD10CM coding rules (including code-first and code-only rules),  the primary diagnosis for home health and  its ICD10-CM code. 
- List of secondary  diagnoses and their ICD10-CM codes

Answer: 
'''


In [ ]:
scenario_number = 49
scenario = """
A 69-year-old man with a years-long history of bipolar disorder attempted to commit suicide by taking all of the remaining contents of a bottle of Advil he kept in his cupboard. Following acute care in the hospital, he's admitted to home health for care of symptoms related to the overdose, which are severe abdominal pain and diarrhea, as well as management of new medications for the bipolar disorder.
 """

text= f'''
The following is a report of a home health patient: 
===
{scenario}
===
The goal is to provide accurate clinical codes for primary and secondary diagnoses in adherence to the Home Health ICD-10-CM diagnosis coding rules.
These rules include 'code-first' rules and rules for etiology that affect which code should be selected as primary as opposed to secondary diagnoses. 
Please provide the following:  
- Bulletpoint list of relevant conditions, in order of relevance, noting new diagnoses that may affect focus of care. 
- Primary focus of care for home health. 
- Adhering to home health coding rules (including code-first / code-only rules and rules on etiologies of symptoms and diseases that affect primary designation), consider what would be the most appropriate primary diagnosis (and ICD10 code) for home health and list the answer. 
- A thorough list of all relevant secondary diagnoses for home health in order of relenvance, and thier ICD10 codes.
- Provide the list of primary and secondary ICD10CM codes and their descriptions in json format.  
- A 
Answer: 
'''


In [ ]:
model = model_ids[0]
result = runtime_prompt_and_response(model_ids[0], text)
outputs.append({'scenario_number': scenario_number, 'prompt':text,'model':model,'result':result})


Here is the list of the patient's relevant conditions in order of relevance:
- Acute ibuprofen toxicity
- Bipolar disorder
- Chronic abdominal pain
- Acute diarrhea 
- Post-suicidal ideation
As the patient has been admitted to home health following acute care in the hospital, the primary focus of care for home health would be to manage symptoms and provide care related to the ibuprofen toxicity, and to help manage the patient's bipolar disorder to prevent similar episodes in the future.
As per Home Health ICD-10-CM diagnosis coding rules, the most appropriate primary diagnosis for home health would be T39.3X5A, "Adverse effect of ibuprofen, accidental (unintentional), initial encounter."
Here is the list of secondary diagnoses for home health in order of relevance:
- Bipolar disorder: F31.81 
- Chronic abdominal pain: R10.3 
- Acute diarrhea: R19.7 
- Post-suicidal ideation: F32.9 ( But F99 is typically recommended by home health coding best practices for post-suicidal ideation instead

In [ ]:
import pandas as pd


In [ ]:
import csv 
# Preprocess the file to fix unescaped quotes
with open('./coding_scenarios.csv', "r") as infile:
    reader = csv.reader(infile)
    header = next(reader)
    s = []
    while True: 
        row = next(reader)
        s.append(row)


StopIteration: 

In [ ]:
header

['index',
 'page_number',
 'scenario_number',
 'scenario',
 'primary_code',
 'primary_desc',
 'secondary_codes',
 'remainder']

In [ ]:
s

[['0',
  '15',
  '1',
  'The patient is being admitted with resolving Acinetobacter baumannii sepsis with acute cystitis.',
  'A41.54',
  'Sepsis due to Acinetobacter baumannii',
  "['N30.00', 'N30.00', 'B96.83']",
  'Secondary:\nN30.00\nAcute cystitis without hematuria\nThe sepsis code is assigned first followed by the local infection code (N30.00). There\nis\nno need to assign the B96.83 code to identify the organism causing the cystitis as the\nsepsis code already identified the organism.\nCopyright © 2023 DecisionHealth\n3'],
 ['1',
  '16',
  '2',
  'The patient is admitted for wound care to a postprocedural intramuscular abscess resulting in sepsis caused by MRSA. IV antibiotics are being administered.',
  'T81.42xA',
  'Infection following a procedure, deep incisional surgical site, initial encounter',
  "['T81.44xA', 'A41.02', 'Z45.2', 'Z79.2', 'T81.44']",
  'Secondary:\nT81.44xA\nSepsis following a procedure, initial encounter\nSecondary:\nA41.02\nMRSA sepsis\nSecondary:\nZ45.2

In [ ]:
len(d)

182

In [ ]:
# Random samping of 10
import random

random_numbers = random.sample(range(183), 10)

In [ ]:
header

['index',
 'page_number',
 'scenario_number',
 'scenario',
 'primary_code',
 'primary_desc',
 'secondary_codes',
 'remainder']

In [ ]:
import textwrap 
evals=[]
random_numbers = random.sample(range(183), 10)
for i in random_numbers: 
    number = s[i][2]
    scenario = s[i][3]
    primary_code = s[i][4]
    primary_desc = s[i][5]
    secondary = s[i][6]
    desc = s[i][7]

    text= f'''
    The following is a report of a home health patient: 
    ===
    {scenario}
    ===
    The goal is to provide accurate clinical codes for primary and secondary diagnoses in adherence to the Home Health ICD-10-CM diagnosis coding rules.
    These rules include 'code-first' rules and rules for etiology that affect which code should be selected as primary as opposed to secondary diagnoses. 
    Please provide the following:  
    - Bulletpoint list of relevant conditions, in order of relevance, noting new diagnoses that may affect focus of care. 
    - Primary focus of care for home health. 
    - List the relevant home health coding rules for diagnosis considerations.  
    - Adhering to home health coding rules (including code-first / code-only rules and rules on etiologies of symptoms and diseases that affect primary designation, whether or not the condition is chronic, resolving, or acute), consider what would be the most appropriate primary diagnosis (and ICD10 code) for home health and list the answer. 
    - A thorough list of all relevant secondary diagnoses for home health in order of relenvance, and thier ICD10 codes.
    - Provide the list of primary and secondary ICD10CM codes and their descriptions in json format.  
    - A 
    Answer: 
    '''
    print(f'\n\n\nScenario: \n {textwrap.fill(scenario, width=100)}')
    model = model_ids[0]
    result = runtime_prompt_and_response(model_ids[0], text)
    print(number)
    print(f'primary code: \n{primary_code}')
    print(f'primary code description: \n{primary_desc}')
    print(f'secondary codes: \n {secondary}')
    print(desc)
    evals.append({'scenario_number': scenario_number, 'prompt':text,'model':model,'result':result, 'answers':[number, primary_code, primary_desc, secondary, desc]})





Scenario: 
 A 67-year-old man is being recerted for another episode of home health to continue wound care to
abdominal surgical wound. He underwent surgery to remove his gallbladder to treat severe
cholecystitis two-and-a-half months ago and the wound became infected into the muscle layer with
cellulitis and E. coli. The cellulitis and infection have now resolved but wound care is still
required. He also has primary osteoarthritis in both knees and chronic systolic heart failure.
 The following is the report with the requested information.  
    ===
    The bulletpoint list of relevant conditions, in order of relevance, noting new diagnoses that may affect focus of care: 
    - Post-operative wound of the abdominal surgical wound (new, affects focus of care). 
    - Osteoarthritis in both knees. 
    - Chronic systolic heart failure. 
    ===
    The primary focus of care for home health is the wound care for the abdominal surgical wound. 
    ===
    The following are relevant home